In [19]:
import os, csv
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras import layers, Input
from keras.optimizers import RMSprop, Adam
from sklearn.model_selection import train_test_split

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import csv
import os,random

from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM, Input
from keras.layers.convolutional import Convolution1D
from keras.layers.core import Dropout, Activation, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import *
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from math import sqrt

import datetime
now = datetime.datetime.now

In [20]:
def process(temp):
    min_len = 3000
    for i in range(11):
        j = 0
        while temp[j][i] != '' and j<len(temp)-1:
            j+=1
        min_len = min(min_len, j);
#     print(min_len)
    return temp[:min_len]

In [21]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.ragged import constant

In [22]:
with open("../po-cf-ex-1-features/E_ID8.csv", 'r') as f:
    temp = list(csv.reader(f, delimiter = ","))
temp = process(temp)
for i in range(len(temp)):
    for j in range(len(temp[0])):
        if temp[i][j]=='':
            print(i,j)
# E_ID8 343,I was Nan

In [23]:
types = {'B_ID':8, 'E_ID':17, 'NE_ID':27, 'S_ID':10, 'P_ID':16}
x_val = []
x_id = []
for ids, vals in types.items():
    for i in range(1,vals+1):
        x_id.append(ids + str(i))
        try: 
            with open("../po-cf-ex-1-features/"+ids+str(i)+".csv", 'r') as f:
                temp = list(csv.reader(f, delimiter = ","))
            temp = process(temp)
            temp = np.asarray(temp)
            temp = temp.astype(np.float64)
        except:
            print("Problem in:", ids, i)
            continue
        x_val.append(temp)
# x_val = np.asarray(sequence.pad_sequences(x_val, maxlen=2500)).astype(np.float64)
print(len(x_val))
for i in range(len(x_val)):
    print(x_val[i].shape)

78
(490, 11)
(889, 11)
(464, 11)
(537, 11)
(981, 11)
(682, 11)
(546, 11)
(865, 11)
(1119, 11)
(784, 11)
(637, 11)
(576, 11)
(420, 11)
(505, 11)
(933, 11)
(587, 11)
(422, 11)
(913, 11)
(838, 11)
(739, 11)
(584, 11)
(553, 11)
(503, 11)
(492, 11)
(536, 11)
(505, 11)
(555, 11)
(897, 11)
(1079, 11)
(905, 11)
(1120, 11)
(612, 11)
(520, 11)
(402, 11)
(492, 11)
(699, 11)
(429, 11)
(299, 11)
(553, 11)
(823, 11)
(988, 11)
(768, 11)
(802, 11)
(883, 11)
(1500, 11)
(562, 11)
(606, 11)
(384, 11)
(762, 11)
(511, 11)
(608, 11)
(798, 11)
(2472, 11)
(684, 11)
(273, 11)
(691, 11)
(1207, 11)
(390, 11)
(959, 11)
(1079, 11)
(988, 11)
(1144, 11)
(581, 11)
(487, 11)
(1206, 11)
(766, 11)
(1528, 11)
(377, 11)
(553, 11)
(444, 11)
(461, 11)
(590, 11)
(484, 11)
(525, 11)
(438, 11)
(643, 11)
(517, 11)
(685, 11)


In [24]:
po_val = []
cf_val = []
for ids in x_id:
    try:
        df = pd.read_excel("../KiMoRe/"+ids+"/Es1/Label/ClinicalAssessment_"+ids+".xlsx")
    except:
        print("problem in: ", ids)
        continue
    df = np.array(df).reshape((16,))
    po_val.append(df[6])
    cf_val.append(df[11])
po_val = np.asarray(po_val).astype(np.float64)
cf_val = np.asarray(cf_val).astype(np.float64)
ts_val = po_val+cf_val
print("shape of CF: ", cf_val.shape)
print("shape of PO: ", po_val.shape)
print("shape of TS: ", ts_val.shape)
# print(po_val)
for i in range(len(ts_val)):
    if np.isnan(ts_val[i]):
        print(i)
print(ts_val)


shape of CF:  (78,)
shape of PO:  (78,)
shape of TS:  (78,)
24
[41.         38.         28.21008112 33.3397327  44.33333333 45.33333333
 40.33333333 44.33333333 48.33333333 49.33333333 42.66666667 44.66666667
 46.         46.66666667 50.         48.66666667 50.         48.33333333
 50.         50.         46.33333333 44.         50.         47.
         nan 39.66666667 41.         44.33333333 46.33333333 40.66666667
 43.66666667 42.         40.         42.         34.         46.
 40.         50.         50.         45.         47.         47.
 38.         50.         46.         46.         43.         41.
 50.         47.         49.         49.         27.         34.
 44.         26.         32.89508774 38.41075339 26.66666667 36.
 26.33333333 33.66666667 48.         27.         50.         14.
 33.         50.         22.         32.         40.         34.3081805
 14.66666667 35.66666667 36.66666667 36.         27.66666667 37.        ]


In [25]:
def jumble_up(val):
    return np.random.permutation(val)

In [26]:
indices = np.arange(len(ts_val))
indices = jumble_up(indices)
print(indices)
temp_x = x_val
temp_ts = ts_val
for i in range(len(ts_val)):
    x_val[i] = temp_x[indices[i]]
    ts_val[i] = temp_ts[indices[i]]

[30 21 42 35 10 37  5 25 28 59  0 58 62 45 57 56 16 60 64 13 70 44 36 29
 76 55 15 32 68 34 74 14 53 38 63 26 18 49 43 65 12 33 31  4 73 52  7 48
  8 20  3 51 66 47 17 46 19 69 67 11 40 41  2 50 22 71 27  1 24 77 54  6
  9 72 75 39 61 23]


In [27]:
# x_val /= np.max(np.abs(x_val))
# ts_val /= np.max(np.abs(ts_val))
max_x_val = -1
max_ts_val = -1
for i in range(len(ts_val)):
    max_x_val = max(max_x_val, np.max(np.abs(x_val[i])))
    max_ts_val = max(max_ts_val, np.max(np.abs(ts_val[i])))
x_val = x_val/max_x_val
ts_val = ts_val/max_ts_val
x_train, x_test, y_train, y_test = train_test_split(x_val,ts_val, test_size=0.3)
print(y_train)

[0.96       0.74       0.52666667 0.8        1.         0.76
 0.85333333 0.81333333 1.         0.52666667 1.         0.73333333
 0.53333333 1.         0.52666667 0.92       0.8        0.71333333
 0.88       0.76821507 0.79333333 0.79333333 0.92       0.85333333
 0.72       0.44       0.96       1.         0.28       0.88
 0.53333333 0.8        0.79333333 0.81333333 0.87333333 0.65790175
 0.54       0.96       1.         0.92       0.8        0.72
 0.72       0.55333333 0.87333333 1.         0.92666667 1.
 1.         0.8        0.98       1.         0.8        1.        ]


<ipython-input-27-81177675099c>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_val = x_val/max_x_val


In [33]:
timesteps = 600 
nr = 78   
n_dim = 11  
dropout_rate = 0.1

In [42]:
def train_generator():
    n=-1
    while n<53:
        n+=1
        xt = np.array(x_train[n])
        yt = np.array(y_train[n])
        yield xt,yt

In [47]:
def Network():
    model = Sequential()
    model.add(Convolution1D(100, 5, padding ='same', strides = 2, input_shape = (None,n_dim)))
    model.add(LeakyReLU())
    model.add(Dropout(dropout_rate))

    model.add(Convolution1D(30, 3, padding ='same', strides = 2))
    model.add(LeakyReLU())
    model.add(Dropout(dropout_rate))

    model.add(Convolution1D(10, 3, padding ='same'))
    model.add(LeakyReLU())
    model.add(Dropout(dropout_rate))
    model.add(Convolution1D(16, 1))
    model.add(Activation('relu'))

    model.add(Convolution1D(8, 1))
    model.add(Activation('relu'))

    # reduce the number of dimensions to the number of classes
    model.add(Convolution1D(1, 1))
    model.add(Activation('sigmoid'))
    print(model.summary())

#     model.add(Flatten())

#     model.add(Dense(200))
#     model.add(LeakyReLU())
#     model.add(Dropout(dropout_rate))

#     model.add(Dense(100))
#     model.add(LeakyReLU())
#     model.add(Dropout(dropout_rate))

#     model.add(Dense(1))
#     model.add(Activation('sigmoid'))


    model.compile(loss='binary_crossentropy', optimizer=Adam())
    
    # Early stopping if the validaton Loss does not decrease for 100 epochs
    early_stopping = EarlyStopping(monitor='val_loss', patience = 100)

    t = now()
    history = model.fit_generator(train_generator(),steps_per_epoch=10, epochs=500, verbose=1,
                     callbacks = [early_stopping])
    print('Training time: %s' % (now() - t))
    
    # Plot the results
    plt.figure(1)
    plt.subplot(221)
    plt.plot(history.history['loss'])
    plt.title('Training Loss')
    plt.subplot(222)
    plt.plot(history.history['val_loss'])
    plt.title('Validation Loss')
    plt.tight_layout()
    plt.show()
    
    # Plot the prediction of the CNN model for the training and validation sets
    pred_train = model.predict(x_train)
    pred_test = model.predict(x_test)

    plt.figure(figsize = (8,8))
    plt.subplot(2,1,1)
    plt.plot(pred_train,'s', color='red', label='Prediction', linestyle='None', alpha = 0.5, markersize=6)
    plt.plot(y_train,'o', color='green',label='Quality Score', alpha = 0.4, markersize=6)
    plt.ylim([-0.1,1.1])
    plt.title('Training Set',fontsize=18)
    plt.xlabel('Sequence Number',fontsize=16)
    plt.ylabel('Quality Scale',fontsize=16)
    plt.legend(loc=3, prop={'size':14}) # loc:position
    plt.subplot(2,1,2)
    plt.plot(pred_test,'s', color='red', label='Prediction', linestyle='None', alpha = 0.5, markersize=6)
    plt.plot(y_test,'o', color='green',label='Quality Score', alpha = 0.4, markersize=6)
    plt.title('Testing Set',fontsize=18)
    plt.ylim([-0.1,1.1])
    plt.xlabel('Sequence Number',fontsize=16)
    plt.ylabel('Quality Score',fontsize=16)
    plt.legend(loc=3, prop={'size':14}) # loc:position
    plt.tight_layout()
#     plt.savefig('../../Results/CNN_Vicon_Scores.png', dpi=300)
    plt.show()
    
    
    # Calculate the cumulative deviation and rms deviation for the validation set
    test_dev = abs(np.squeeze(pred_test)-y_test)
    # Cumulative deviation
    mean_abs_dev = np.mean(test_dev)
    # RMS deviation
    rms_dev = sqrt(mean_squared_error(pred_test, y_test))
    print('Mean absolute deviation:', mean_abs_dev)
    print('RMS deviation:', rms_dev)
    
    return mean_abs_dev, rms_dev

In [48]:
Mean_abs_dev, RMS_dev  = Network()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_42 (Conv1D)           (None, None, 100)         5600      
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, None, 100)         0         
_________________________________________________________________
dropout_21 (Dropout)         (None, None, 100)         0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, None, 30)          9030      
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, None, 30)          0         
_________________________________________________________________
dropout_22 (Dropout)         (None, None, 30)          0         
_________________________________________________________________
conv1d_44 (Conv1D)           (None, None, 10)        

ValueError: in user code:

    /home/yash/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/yash/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/yash/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/yash/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/yash/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/yash/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py: